# Nearest‑Neighbor Workflow (SPAC)

This **three‑step notebook** reproduces the workflow you shared:

1. **Nearest‑neighbor calculation** – `calculate_nearest_neighbor`  
2. **Assign *pin colors*** for consistent phenotype colors across all plots  
3. **Visualize distance distributions** – `visualize_nearest_neighbor`

⚠️ If you *already* have an `AnnData` object named `adata` in your notebook session, simply **skip** the (optional) “Load data” code cell below.

## 📦 Imports

In [ ]:
import pickle
import matplotlib.pyplot as plt
import seaborn as sns  # only used for palette generation

# SPAC core functions
from spac.spatial_distance import calculate_nearest_neighbor, visualize_nearest_neighbor


## 0️⃣ (Optional) Load input `AnnData`

In [ ]:
# ---------------------------------------------------------------------------
# If `adata` is already defined, comment‑out or skip this cell.
# ---------------------------------------------------------------------------
INPUT_FILE = "transform_output.pickle"

with open(INPUT_FILE, "rb") as f:
    adata = pickle.load(f)

print(f"Loaded AnnData with {adata.n_obs:,} cells and {adata.n_vars:,} variables.")


## 1️⃣ Calculate nearest neighbors

In [ ]:
# ── user parameters ──────────────────────────────────────────────────────────
annotation = "renamed_clusters"
spatial_associated_table = "spatial"
imageid = None              # Column name if multiple images; else None
distance_key = "spatial_distance"
# ─────────────────────────────────────────────────────────────────────────────

calculate_nearest_neighbor(
    adata=adata,
    annotation=annotation,
    spatial_associated_table=spatial_associated_table,
    imageid=imageid,
    label=distance_key,
)

print(f"NN distances stored under adata.obsm['{distance_key}'] "
      f"({adata.obsm[distance_key].shape}).")


## 2️⃣ Assign *pin colors*

Interactive spatial viewers in SPAC use a **pin‑color map** so that every
phenotype keeps the **same color** across figures.  
The helper below builds such a map if one is not present.

In [ ]:
def set_pin_colors(adata, annotation, palette='tab20'):
    """Create a phenotype→color dict and store it in `adata.uns['pin_colors']`."""
    if 'pin_colors' in adata.uns:
        print("Pin colors already exist → skipping.")
        return

    phenotypes = sorted(adata.obs[annotation].unique())
    n = len(phenotypes)
    colors = sns.color_palette(palette, n).as_hex()

    adata.uns['pin_colors'] = dict(zip(phenotypes, colors))
    print(f"Stored {n} colors under adata.uns['pin_colors'].")

# ---------------------------------------------------------------------------
set_pin_colors(adata, annotation)


## 3️⃣ Visualize nearest‑neighbor distances

In [ ]:
# ── visualization parameters ────────────────────────────────────────────────
image_id = None
method = "numeric"             # 'numeric' or 'distribution'
plot_type = "violin"
source_label = "Follicular helper T cells (Tfh)"
target_label = ("Proliferative B cells, NK cells, CD8 T cells, "
                "Regulatory T cells, Follicular helper T cells (Tfh)")
log_scale = True
facet_plot = False

fig_width, fig_height, fig_dpi = 10, 6, 300
font_size = 10
fig_title = (f'Nearest‑Neighbor Distance Distribution\n'
             f'Measured from "{source_label}"')
# ─────────────────────────────────────────────────────────────────────────────

distance_to = [x.strip() for x in target_label.split(",")] if target_label else None
plt.rcParams.update({'font.size': font_size})

extra_kwargs = {"palette": adata.uns.get('pin_colors', None)}
# Faceting options
if facet_plot:
    extra_kwargs.update({"col_wrap": 2, "hue": "group"})

result = visualize_nearest_neighbor(
    adata=adata,
    annotation=annotation,
    spatial_distance=distance_key,
    distance_from=source_label,
    distance_to=distance_to,
    method=method,
    plot_type=plot_type,
    stratify_by=image_id,
    facet_plot=facet_plot,
    log=log_scale,
    **extra_kwargs,
)

def _set_fig_props(fig):
    fig.set_size_inches(fig_width, fig_height)
    fig.set_dpi(fig_dpi)
    fig.tight_layout(rect=[0, 0, 1, 0.90])
    fig.suptitle(fig_title, fontsize=font_size + 4, weight='bold')

if isinstance(result["fig"], (list, tuple)):
    for f in result["fig"]:
        _set_fig_props(f)
else:
    _set_fig_props(result["fig"])

plt.show()


---

*Notebook last built 2025‑07‑09 20:45 UTC.*